Don't modify ID's

In [1]:
import os
import pandas as pd
import numpy as np
import re

In [2]:
DATA_DIR = "/Users/kendra/Data/IMDB/2024-10-05"

## name

In [3]:
file_path = os.path.join(DATA_DIR, "name.basics.tsv")

In [4]:
df1 = pd.read_csv(file_path, sep='\t')

In [5]:
df1.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


In [6]:
ignore_cols = ['primaryProfession', 'knownForTitles']

rename_dict = {
    'nconst': 'nameId:ID(Person)', #'nameId:ID', #
    'primaryName': 'name',
    'birthYear': 'birthYear:int',
    'deathYear': 'deathYear:int'
}

# for col in ignore_cols:
#     rename_dict[col] = f'{col}:IGNORE'
drop_cols = ignore_cols

In [7]:
df1['is_actor'] = np.where(
    (df1['primaryProfession'].str.contains('actor')) | (df1['primaryProfession'].str.contains('actress')),
    1, 0)
df1['is_director'] = np.where(
    (df1['primaryProfession'].str.contains('director')),
    1, 0)  

df1_filt = df1[(df1['is_actor'] == 1) | (df1['is_director'] == 1)]

In [8]:
n = len(df1)
nf1 = len(df1_filt)
nf1, nf1/n

(6019760, 0.43474919893183206)

In [9]:
df1_filt = df1_filt.drop(columns=['is_actor', 'is_director'])# 'primaryProfession:IGNORE', 'knownForTitles:IGNORE'])

Remove people who don't have a birth year (imaginary characters?)

In [10]:
df1_filt = df1_filt[df1_filt['birthYear'] != '\\N']

n_filt2 = len(df1_filt)
n_filt2, n_filt2/nf1

(434118, 0.07211549962124736)

In [11]:
# replace '\N' with nulls
df1_filt['deathYear'] = np.where(df1_filt['deathYear'] == '\\N', np.nan, df1_filt['deathYear'])

In [12]:
df1_rename = (df1_filt
              .rename(columns=rename_dict)
              .drop(columns=ignore_cols)
             )

In [13]:
df1_rename[':LABEL'] = 'Person'

In [14]:
df1_rename.sample(10)

,nameId:ID,name,birthYear:int,deathYear:int,:LABEL
28900,nm0030157,Jack Anker,1925,2002,Person
5338294,nm1478052,Kathy Danzer,1951,NaN,Person
490144,nm0519247,Karin Longacre,1951,2020,Person
884741,nm0941168,Richard Woolf,1979,NaN,Person
817815,nm0869336,Jerry Toth,1928,1999,Person
124322,nm0130717,Florencio Calpe,1916,1989,Person
7191553,nm1947433,Adam Sztaba,1975,NaN,Person
12356544,nm8151982,Duling Chen,1993,NaN,Person
96323,nm0101076,Lally Bowers,1917,1984,Person
7305544,nm2076567,Tommi Evilä,1980,NaN,Person


In [15]:
len(df1_rename)

434118

In [16]:
# df1_rename.to_csv(os.path.join(DATA_DIR, "filt.name.basics.tsv"), sep='\t', header=True, index=False)
df1_rename.to_csv(os.path.join(DATA_DIR, "names.tsv"), sep='\t', header=True, index=False)

In [17]:
del df1, df1_filt, df1_rename

# title basics

In [18]:
file_path = os.path.join(DATA_DIR, "title.basics.tsv")
df = pd.read_csv(file_path, sep='\t') #, low_memory=False)

/var/folders/mp/z98sg8jn1q3bx499xmgkf_040000gn/T/ipykernel_30477/2806677326.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t') #, low_memory=False)


In [19]:
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [20]:

rename_dict = {
    'tconst': 'titleId:ID(Movie)', # 'titleId:ID', #
    'primaryTitle': 'title',
    'startYear': 'year:int',
    'runtimeMinutes': 'runtimeMinutes:int'
}

# we'll drop the rest
drop_cols = ['isAdult', 'originalTitle', 'endYear', 'titleType']

In [21]:
n = len(df)

df_filt = df[df['isAdult'] == 0]

n_filt = len(df_filt)
n_filt, n_filt/n

(10706453, 0.962146450179395)

In [22]:
df_movies = df_filt[df_filt['titleType'] == 'movie']
n_m = len(df_movies)
n_m, n_m/n_filt

(681857, 0.06368654492762449)

In [23]:
# replace '\N' with nulls
df_movies['startYear'] = np.where(df_movies['startYear'] == '\\N', np.nan, df_movies['startYear'])
df_movies['runtimeMinutes'] = np.where(df_movies['runtimeMinutes'] == '\\N', np.nan, df_movies['runtimeMinutes'])

/var/folders/mp/z98sg8jn1q3bx499xmgkf_040000gn/T/ipykernel_30477/3649384730.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['startYear'] = np.where(df_movies['startYear'] == '\\N', np.nan, df_movies['startYear'])
/var/folders/mp/z98sg8jn1q3bx499xmgkf_040000gn/T/ipykernel_30477/3649384730.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['runtimeMinutes'] = np.where(df_movies['runtimeMinutes'] == '\\N', np.nan, df_movies['runtimeMinutes'])


In [24]:
df_rename = (df_movies
             .drop(columns=drop_cols)
             .rename(columns=rename_dict)
            )

In [25]:
df_rename[':LABEL'] = 'Movie'

In [26]:
df_rename.sample(10)

,titleId:ID,title,year:int,runtimeMinutes:int,genres,:LABEL
11097007,tt9849964,All of Them Are Troublesome Girls!,2019,79,"Comedy,Romance",Movie
3901677,tt1538901,Next Year Country,2010,57,"Documentary,News,Western",Movie
6903109,tt28650534,Song of the Body,NaN,90,"Drama,Musical",Movie
5752943,tt2248675,"Ang leon, ang tigre, at ang alamid",1979,113,Action,Movie
8301264,tt3635544,Imagining Indians,1992,60,"Documentary,History",Movie
9006376,tt5232126,Mestisang Bangus,1963,NaN,Comedy,Movie
6341841,tt2671344,Birbal My Brother,1973,105,Drama,Movie
5543160,tt2165739,"Easy come, easy go",2008,94,Crime,Movie
3027605,tt13776786,Person Woman Man Camera TV,2022,77,Drama,Movie
26253,tt0026730,The Morals of Marcus,1935,75,Comedy,Movie


In [27]:
len(df_rename)

681857

In [28]:
# df_rename.to_csv(os.path.join(DATA_DIR, "filt.titles.basic.tsv"), sep="\t", header=True, index=False)
df_rename.to_csv(os.path.join(DATA_DIR, "movies.tsv"), sep="\t", header=True, index=False)

In [29]:
del df, df_movies, df_rename

# check principal (relationship?)

In [ ]:
file_path = os.path.join(DATA_DIR, "filt.title.principals.tsv")
df = pd.read_csv(file_path, sep='\t', nrows=10000)

In [ ]:
df.head()

In [ ]:
df.info()

In [35]:
file_path = os.path.join(DATA_DIR, "roles.tsv")
df = pd.read_csv(file_path, sep='\t', header=None)

In [36]:
df.head()

,0,1,2
0,tt0000001,nm0005690,director
1,tt0000002,nm0721526,director
2,tt0000003,nm0721526,director
3,tt0000004,nm0721526,director
4,tt0000005,nm0443482,actor


In [37]:
df.columns = ['movieId', 'personId', 'rel_type']

In [32]:
len(df)

44521804

In [39]:
len(df[df['movieId'].isna()])

0

In [40]:
len(df[df['personId'].isna()])

0

In [41]:
len(df[df['rel_type'].isna()])

0

In [42]:
name_df = pd.read_csv(os.path.join(DATA_DIR, 'names.tsv'), sep='\t')

In [43]:
movies_df = pd.read_csv(os.path.join(DATA_DIR, 'movies.tsv'), sep='\t')

In [44]:
names = name_df['nameId:ID(Person)'].unique()
movies = movies_df['titleId:ID(Movie)'].unique()

In [45]:
df_filt = df[
    (df['personId'].isin(names)) & (df['movieId'].isin(movies))
]

In [46]:
len(df), len(df_filt)

(44521805, 2166612)

In [50]:
df_filt.columns = [":END_ID(Movie)", ":START_ID(Person)", ":TYPE"]

In [51]:
df_filt.head()

,:END_ID(Movie),:START_ID(Person),:TYPE
12,tt0000009,nm0063086,actor
13,tt0000009,nm0183823,actor
14,tt0000009,nm1309758,actor
15,tt0000009,nm0085156,director
187,tt0000147,nm0714557,director


In [52]:
df_filt.to_csv(os.path.join(DATA_DIR, "roles.filt.tsv"), sep="\t", header=True, index=False)

# dummy data

In [ ]:
names_df = pd.read_csv(os.path.join(DATA_DIR, 'filt.name.basics.tsv'), sep='\t')

In [ ]:
names_df.head()

In [ ]:
movies_df = pd.read_csv(os.path.join(DATA_DIR, 'filt.titles.basic.tsv'), sep='\t')

In [ ]:
movies_df.head()

In [ ]:
movies_df[movies_df['startYear'] > 1980].sample(5)

In [ ]:
movies_df[movies_df['startYear'] > 1980].sample(5)

In [ ]:
names_df[names_df['birthYear'] > 1960].sample(5)